## Otter Interview Project - Analysis on restaurants with delivery services in the US ##

## 1. Initial Setup ##

#### 1.1 Library required for the analysis ####

In [547]:
library(tidyr)
library(dplyr)
library(ggmap)
library(psych)
library(maps)
library(ggplot2)
library(xray)
library(data.table)
library(finalfit)
library(tidygeocoder)
library(summarytools)

#### 1.2 Data Preparation ####

- Set the working directory
- Read the file
- Ensure the dataset is prepared and converted into proper dataframe format

In [548]:
## setwd("/Users/tonychu/SynologyDrive/Resume:Application related document/Otter")
df <- read.table("css_public_all_ofos_locations.csv",header=TRUE, sep='\001',na.strings ='\\N', quote = "", fill=T)

In [549]:
head(df)
dim(df)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
1,9744,Loco Coco,caviar,,40.765004,-73.965961,,US,TRUE,loco_coco,,,NA
2,4903,Musubi,caviar,,45.504641,-122.643806,,US,FALSE,musubi,,,NA
3,4723,The Bakers' Lounge,caviar,,38.924559,-76.990115,,US,FALSE,the_bakers_lounge,,,NA
4,4128,Parm - Battery Park City,caviar,,40.71418,-74.015568,,US,TRUE,parm_-_battery_park_city,,,NA
5,5690,Derek Test Merchant,caviar,,1,2,,US,FALSE,derek_test_merchant,,,NA
6,2827,The Plant Cafe - California & Pine (FiDi),caviar,,37.792829,-122.397881,,US,FALSE,the_plant_cafe_-_california_pine_fidi,,,NA


[1] 644089     13

##### 1.3 Convert the attributes into the proper data type #####

- Change the data type of **restaurant_id, latitude, and longitude** into numeric variables

In [550]:
num_col = c("restaurant_id","latitude", "longitude","delivery_radius")
df[ , num_col] <- apply(df[ , num_col], 2, function(x) as.numeric(as.character(x)))
head(df)

Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”


,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
1,9744,Loco Coco,caviar,,40.76500,-73.96596,,US,TRUE,loco_coco,,NA,NA
2,4903,Musubi,caviar,,45.50464,-122.64381,,US,FALSE,musubi,,NA,NA
3,4723,The Bakers' Lounge,caviar,,38.92456,-76.99012,,US,FALSE,the_bakers_lounge,,NA,NA
4,4128,Parm - Battery Park City,caviar,,40.71418,-74.01557,,US,TRUE,parm_-_battery_park_city,,NA,NA
5,5690,Derek Test Merchant,caviar,,1.00000,2.00000,,US,FALSE,derek_test_merchant,,NA,NA
6,2827,The Plant Cafe - California & Pine (FiDi),caviar,,37.79283,-122.39788,,US,FALSE,the_plant_cafe_-_california_pine_fidi,,NA,NA


## 2. Data Exploration and data cleaning ##
    

- Look at the summary of the dataframe consisting of: variable names and labels, factor levels, frequencies or numerical summary statistics, and valid/missing observations information.
- There are 644,089 entries in total
- Seems like plenty of the attributes require further examinations, and below are some of the questions need to be addressed regarding data exploration:
    - Is there any missing restaurant ID, and would missing restaurant ID impact the analysis?
    - How many delivery platforms and sub platforms are there?
    - Are there restaurants with missing coordinates? Can I identify duplicated restaurants with no coordinates?
    - How many restaurants are missing the city attribute? How do I extract city information without city attribute? Reverse check-up using the coordinates?
    - Are we certain all restaurants are all located in the US?
    - How do I standardize restaurant names and how do I remove duplicated entries 

In [551]:
subset(dfSummary(df), select=-Graph)

,No,Variable,Stats / Values,Freqs (% of Valid),text.graph,Valid,Missing
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,restaurant_id\ [numeric],Mean (sd) : 237866.8 (187782.5)\ min < med < max:\ 1 < 216563 < 997804\ IQR (CV) : 249348 (0.8),402721 distinct values,:\ : \ \ . .\ : : : :\ : : : :\ : : : : : .,467907\ (72.6%),176182\ (27.4%)
2,2,name\ [character],1\. Subway\ 2\. Starbucks\ 3\. McDonald's\ 4\. Taco Bell\ 5\. Burger King\ 6\. 7-Eleven\ 7\. KFC\ 8\. Walgreens\ 9\. Dunkin' Donuts\ 10\. Wendy's\ [ 418989 others ],\ 8554 ( 1.3%)\ \ 8047 ( 1.2%)\ \ 6951 ( 1.1%)\ \ 4997 ( 0.8%)\ \ 3645 ( 0.6%)\ \ 3398 ( 0.5%)\ \ 2998 ( 0.5%)\ \ 2770 ( 0.4%)\ \ 2729 ( 0.4%)\ \ 2092 ( 0.3%)\ \597908 (92.8%),\ \ \ \ \ \ \ \ \ \ IIIIIIIIIIIIIIIIII,644089\ (100.0%),0\ (0.0%)
3,3,platform\ [character],"1\. doordash\ 2\. postmates\ 3\. grubhub\ 4\. ubereats\ 5\. delivery.com\ 6\. caviar\ 7\. (Empty string)\ 8\. ·[""""open: 12:00\ 9\. ·[]\ 10\. ·[""""open: 11:00\ [ 37 others ]",\353739 (54.9%)\ \110165 (17.1%)\ \ 68029 (10.6%)\ \ 65422 (10.2%)\ \ 34729 ( 5.4%)\ \ 9964 ( 1.5%)\ \ 1845 ( 0.3%)\ \ 44 ( 0.0%)\ \ 28 ( 0.0%)\ \ 20 ( 0.0%)\ \ 104 ( 0.0%),IIIIIIIIII \ III \ II \ II \ I \ \ \ \ \ \,644089\ (100.0%),0\ (0.0%)
4,4,sub_platform\ [character],"1\. doordash\ 2\. restaurant\ 3\. grubhub\ 4\. ubereats\ 5\. (Empty string)\ 6\. ·close: 21:00""""]\ 7\. ·close: 20:00""""]\ 8\. ·[]\ 9\. ·close: 22:00""""]\ 10\. ·close: 00:00""""]\ [ 21 others ]",\353739 (54.9%)\ \110165 (17.1%)\ \ 68029 (10.6%)\ \ 65422 (10.2%)\ \ 46538 ( 7.2%)\ \ 45 ( 0.0%)\ \ 28 ( 0.0%)\ \ 23 ( 0.0%)\ \ 15 ( 0.0%)\ \ 14 ( 0.0%)\ \ 71 ( 0.0%),IIIIIIIIII \ III \ II \ II \ I \ \ \ \ \ \,644089\ (100.0%),0\ (0.0%)
5,5,latitude\ [numeric],Mean (sd) : 37 (8.5)\ min < med < max:\ -90 < 38.5 < 555\ IQR (CV) : 7 (0.2),543253 distinct values,\ \ :\ \ \ : :\ \ \ : :\ \ \ : :\ \ \ : :,642047\ (99.7%),2042\ (0.3%)
6,6,longitude\ [numeric],Mean (sd) : -94 (19.1)\ min < med < max:\ -158.3 < -87.8 < 555\ IQR (CV) : 37.7 (-0.2),540387 distinct values,: .\ : :\ : :\ : :\ : :,642047\ (99.7%),2042\ (0.3%)
7,7,city\ [character],1\. (Empty string)\ 2\. Los Angeles\ 3\. New York\ 4\. Houston\ 5\. Chicago\ 6\. Miami\ 7\. Philadelphia\ 8\. San Diego\ 9\. San Francisco\ 10\. Orlando\ [ 3050 others ],\588165 (91.3%)\ \ 1831 ( 0.3%)\ \ 1630 ( 0.3%)\ \ 1225 ( 0.2%)\ \ 1163 ( 0.2%)\ \ 1114 ( 0.2%)\ \ 893 ( 0.1%)\ \ 808 ( 0.1%)\ \ 790 ( 0.1%)\ \ 739 ( 0.1%)\ \ 45731 ( 7.1%),IIIIIIIIIIIIIIIIII \ \ \ \ \ \ \ \ \ \ I,644089\ (100.0%),0\ (0.0%)
8,8,country\ [character],"1\. US\ 2\. (Empty string)\ 3\. ·close: 21:00""""]\ 4\. ·close: 20:00""""]\ 5\. FALSE\ 6\. ·close: 22:00""""]\ 7\. ·close: 00:00""""]\ 8\. ·close: 20:30""""]\ 9\. ·close: 23:00""""]\ 10\. ·close: 21:30""""]\ [ 18 others ]",\642046 (99.7%)\ \ 1863 ( 0.3%)\ \ 40 ( 0.0%)\ \ 29 ( 0.0%)\ \ 23 ( 0.0%)\ \ 18 ( 0.0%)\ \ 17 ( 0.0%)\ \ 7 ( 0.0%)\ \ 7 ( 0.0%)\ \ 6 ( 0.0%)\ \ 33 ( 0.0%),IIIIIIIIIIIIIIIIIII \ \ \ \ \ \ \ \ \ \,644089\ (100.0%),0\ (0.0%)
9,9,active\ [character],"1\. TRUE\ 2\. FALSE\ 3\. (Empty string)\ 4\. ·close: 21:00""""]\ 5\. ·close: 20:00""""]\ 6\. ·close: 22:00""""]\ 7\. ·close: 00:00""""]\ 8\. ·close: 23:00""""]\ 9\. ·[]]""\ 10\. 40\ [ 21 others ]",\567052 (88.0%)\ \ 74802 (11.6%)\ \ 2065 ( 0.3%)\ \ 35 ( 0.0%)\ \ 25 ( 0.0%)\ \ 20 ( 0.0%)\ \ 19 ( 0.0%)\ \ 13 ( 0.0%)\ \ 7 ( 0.0%)\ \ 7 ( 0.0%)\ \ 44 ( 0.0%),IIIIIIIIIIIIIIIII \ II \ \ \ \ \ \ \ \ \,644089\ (100.0%),0\ (0.0%)


##### 2.1 Data Cleaning by restaurant ID #####

- By examining restaurants with no restaurant ID, seems like i am still able to find out the information that I required without restaurant ID.

In [552]:
head(subset(df,is.na(restaurant_id)),20)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
34795,NA,,delivery.com,,NA,NA,,US,FALSE,,,NA,NA
94112,NA,Hapa Sushi LoDo,postmates,restaurant,39.74985,-104.99994,,US,TRUE,hapa_sushi_lodo,,NA,NA
94113,NA,Fat Sal's,postmates,restaurant,34.06253,-118.44805,,US,TRUE,fat_sal_s,,NA,NA
94114,NA,Monk's Kettle,postmates,restaurant,37.76489,-122.42291,,US,TRUE,monk_s_kettle,,NA,NA
94115,NA,Firehouse Subs,postmates,restaurant,41.70156,-83.65030,,US,TRUE,firehouse_subs,,NA,NA
94116,NA,Carl's Jr/Green Burrito,postmates,restaurant,33.81533,-116.40101,,US,FALSE,carl_s_jr_green_burrito,,NA,NA
94117,NA,Dunkin' Donuts,postmates,restaurant,40.71571,-73.59847,,US,TRUE,dunkin_donuts,,NA,NA
94118,NA,Subway,postmates,restaurant,39.81755,-86.32782,,US,TRUE,subway,,NA,NA
94119,NA,7-Eleven,postmates,restaurant,38.96194,-76.86317,,US,FALSE,7-eleven,,NA,NA


##### 2.2 Data cleaning by restaurant name #####

- Change all restaurant names to lower case to simplify data deduplication
- Let's examine the name of the restaurant. Apparently, only 65 restaurants have no proper name entry nor other information. So they can comfortably be removed as it only made up small portion of the overall dataset.

In [553]:
df$name <- tolower(df$name)

- Create a subset of entries with no restaurant names

In [564]:
head(subset(df, name==""))
no_name <- subset(df, name=="")
dim(no_name)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
675,4075,,,,NA,NA,,,,,,NA,NA
27428,64030,,,,NA,NA,,,,,,NA,NA
29810,646,,,,NA,NA,,,,,,NA,NA
29929,63819,,,,NA,NA,,,,,,NA,NA
34795,NA,,delivery.com,,NA,NA,,US,FALSE,,,NA,NA
41511,72462,,,,NA,NA,,,,,,NA,NA


[1] 65 13

- Assuming due to system error or manual entries, 170 of the entries have information recorded that cannot be contributed to the analysis as well. So we will remove them also.
- By further examining the name attribute, some entries have "duplicate","training","do not use","do not enable","churned","closed" in their names, thus I automatically assume this is mainly due to manual entry or name given by system due to specific reasons. There are 5,839 entries like this, and I decide to remove them because they also only made up a very small portion of the dataset. 

In [565]:
head(filter(df, grepl('duplicate|training|do not use|do not enable|churned|closed|close:|open:|usd', name)),10)
bad_entry <- filter(df, grepl('duplicate|training|do not use|do not enable|churned|closed|close:|open:|usd', name))
dim(bad_entry)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
1,6878,chen's chinese & cuisine {do not use},caviar,,41.8889,-87.6330,,US,FALSE,chen_s_chinese_cuisine_do_not_use,,NA,NA
2,8216,king��s road cafe - do not enable,caviar,,34.0762,-118.3723,,US,FALSE,king_s_road_cafe_-_do_not_enable,,NA,NA
3,73588,andrew's diner - using for training,delivery.com,,40.5383,-74.1490,,US,FALSE,andrew_s_diner_-_using_for_training,,NA,NA
4,88006,oob - closed - lee chi,delivery.com,,40.7769,-73.9525,,US,FALSE,oob_-_closed_-_lee_chi,,NA,NA
5,5074,yucatan taco stand- do not enable,caviar,,32.8153,-96.7703,,US,FALSE,yucatan_taco_stand-_do_not_enable,,NA,NA
6,3700,"[do not enable, bad duplicate] meehan's (sandy sprigs)",caviar,,33.8395,-84.3771,,US,FALSE,do_not_enable_bad_duplicate_meehan_s_sandy_sprigs,,NA,NA
7,601,tiffin bistro duplicate,caviar,,39.9347,-75.1622,,US,FALSE,tiffin_bistro_duplicate,,NA,NA
8,803,pure fare - south street duplicate,caviar,,39.9442,-75.1692,,US,FALSE,pure_fare_-_south_street_duplicate,,NA,NA
9,725,phuong thao [do not enable],caviar,,37.3769,-122.0305,,US,FALSE,phuong_thao_do_not_enable,,NA,NA


[1] 6009   13

##### 2.3 Data cleaning by coordinates #####

- There are 2,042 entries with no coordinates available
- There are 1,759 entries with wrong coordinates that point to either the North Pole, South Pole, or other extreme locations
- So it is safe to say that we can remove these entries as we have no ways to find out where their exact locations are
- Coordinates should be round up the 4th decimal place since each degree in longitude and latitude is around 111 KM. It is reasonable to assume that there can be different restaurants 10 meter apart on the map as 10 meter equates to 0.0001 degree in terms of coordinates.

In [566]:
df$latitude <- round(df$latitude,4)
df$longitude <- round(df$longitude,4)

In [567]:
head(subset(df, is.na(longitude) & is.na(latitude)),10)
no_cor <- subset(df, is.na(longitude) & is.na(latitude))
dim(no_cor)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
675,4075,,,,NA,NA,,,,,,NA,NA
1927,86915,el sazon de las americas,,,NA,NA,,,,,,NA,NA
2092,8142,ten sushi,,,NA,NA,,,,,,NA,NA
4167,54962,al's,,,NA,NA,,,,,,NA,NA
4387,61610,fake restaurant - reviewer,,,NA,NA,,,,,,NA,NA
4950,87158,mocha's cafe & grill,,,NA,NA,,,,,,NA,NA
5908,56614,chili's grill & bar,,,NA,NA,,,,,,NA,NA
6005,30732,extreme pizza: template (,,,NA,NA,,,,,,NA,NA
6218,35120,soho thai,,,NA,NA,,,,,,NA,NA


[1] 2042   13

In [568]:
head(subset(df, ((abs(latitude) == 90 & longitude ==0 ) | (abs(latitude) <= 1 | abs(longitude) <= 1 ) | (abs(latitude) > 90 | abs(longitude) > 180))),10)
wrong_cor <- subset(df, ((abs(latitude) == 90 & longitude ==0 ) | (abs(latitude) <= 1 | abs(longitude) <= 1 ) | (abs(latitude) > 90 | abs(longitude) > 180)))
dim(wrong_cor)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
5,5690,derek test merchant,caviar,,1,2,,US,FALSE,derek_test_merchant,,NA,NA
61,59739,bequ juice,delivery.com,,-90,0,,US,FALSE,bequ_juice,,NA,NA
65,63820,gino's east,delivery.com,,-90,0,,US,FALSE,gino_s_east,,NA,NA
101,64603,sori sushi,delivery.com,,-90,0,,US,FALSE,sori_sushi,,NA,NA
138,5648,jessica hui placeholder,caviar,,0,0,,US,FALSE,jessica_hui_placeholder,,NA,NA
161,1526,the halal guys,caviar,,0,0,,US,FALSE,the_halal_guys,,NA,NA
167,9812,the french bakery - 112th ave [disabled],caviar,,1,-1,,US,FALSE,the_french_bakery_-_112th_ave_disabled,,NA,NA
202,63305,ezras food,delivery.com,,-90,0,,US,FALSE,ezras_food,,NA,NA
213,57676,cupping room cafe,delivery.com,,-90,0,,US,FALSE,cupping_room_cafe,,NA,NA


[1] 1763   13

##### 2.4 Data cleaning by platform status #####

- There are 2,065 entries with no delivery platform status available


In [569]:
head(subset(df, active == ""))
null_active <- subset(df, active == "")
dim(null_active)

,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
675,4075,,,,NA,NA,,,,,,NA,NA
1927,86915,el sazon de las americas,,,NA,NA,,,,,,NA,NA
2092,8142,ten sushi,,,NA,NA,,,,,,NA,NA
4167,54962,al's,,,NA,NA,,,,,,NA,NA
4387,61610,fake restaurant - reviewer,,,NA,NA,,,,,,NA,NA
4950,87158,mocha's cafe & grill,,,NA,NA,,,,,,NA,NA


[1] 2065   13

##### 2.5 Data cleaning by removing unneeded subsets #####

- After removing the subsets, there are still 639,998 entries left

In [ ]:
no_name <- subset(df, name=="")
bad_entry <- filter(df, grepl('duplicate|training|do not use|do not enable|churned|closed|close:|open:|usd', name))
no_cor <- subset(df, is.na(longitude) & is.na(latitude))
wrong_cor <- subset(df, ((abs(latitude) == 90 & longitude ==0 ) | (abs(latitude) <= 1 | abs(longitude) <= 1 ) | (abs(latitude) > 90 | abs(longitude) > 180)))
null_active <- subset(df, active == "")

In [570]:
df_in_work <- df
df_in_work <- anti_join(df_in_work, no_name)
df_in_work <- anti_join(df_in_work, bad_entry)
df_in_work <- anti_join(df_in_work, no_cor)
df_in_work <- anti_join(df_in_work, wrong_cor)
df_in_work <- anti_join(df_in_work, null_active)

dim(df_in_work)

Joining, by = c("restaurant_id", "name", "platform", "sub_platform", "latitude", "longitude", "city", "country", "active", "standardized_name", "restaurant_chain", "delivery_radius", "geom")

Joining, by = c("restaurant_id", "name", "platform", "sub_platform", "latitude", "longitude", "city", "country", "active", "standardized_name", "restaurant_chain", "delivery_radius", "geom")

Joining, by = c("restaurant_id", "name", "platform", "sub_platform", "latitude", "longitude", "city", "country", "active", "standardized_name", "restaurant_chain", "delivery_radius", "geom")

Joining, by = c("restaurant_id", "name", "platform", "sub_platform", "latitude", "longitude", "city", "country", "active", "standardized_name", "restaurant_chain", "delivery_radius", "geom")

Joining, by = c("restaurant_id", "name", "platform", "sub_platform", "latitude", "longitude", "city", "country", "active", "standardized_name", "restaurant_chain", "delivery_radius", "geom")

Joining, by = c("restaurant_id", "name",

[1] 634255     13

## 3. Feature Engineering ##

##### 3.1 Restaurant name exploration and cleaning #####

- Separate names with branch names in parenthesis into two columns "restaurant_chain" & "branch"

In [572]:
df_in_work$name <- gsub("\\(|\\[", "- ", df_in_work$name)
df_in_work$name <- gsub("\\)|\\]|\\�", "", df_in_work$name)


In [573]:
df_in_work <- separate(df_in_work,name,into = c("restaurant_chain","branch"), sep=" - ", extra="merge")

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 430929 rows [1, 2, 3, 6, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 27, 28, 30, ...].”


In [574]:
head(df_in_work)
dim(df_in_work)

,restaurant_id,restaurant_chain,branch,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,delivery_radius,geom
,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>
1,9744,loco coco,NA,caviar,,40.7650,-73.9660,,US,TRUE,loco_coco,NA,NA
2,4903,musubi,NA,caviar,,45.5046,-122.6438,,US,FALSE,musubi,NA,NA
3,4723,the bakers' lounge,NA,caviar,,38.9246,-76.9901,,US,FALSE,the_bakers_lounge,NA,NA
4,4128,parm,battery park city,caviar,,40.7142,-74.0156,,US,TRUE,parm_-_battery_park_city,NA,NA
5,2827,the plant cafe,california & pine - fidi,caviar,,37.7928,-122.3979,,US,FALSE,the_plant_cafe_-_california_pine_fidi,NA,NA
6,4850,minton's,NA,caviar,,40.8047,-73.9523,,US,FALSE,minton_s,NA,NA


[1] 634255     13

- Create a list of franchise restaurant by grouping restaurant names with over 100 branches by frequencies to find unique restaurant names
- Assuming restaurants with less than 100 stores are not considered as franchises
- Run a for loop and rename restaurants that have similar names to make sure the list of restaurant names are more uniform
- Create another column type, and name those with over 100 appearances "Franchises", and the ones with less than 100 stores "SMBs"

In [575]:
franchises <- group_by(df_in_work, restaurant_chain) %>% summarize(n = n()) %>% arrange(desc(n)) %>% subset(n>100)
df_in_work$type <- ""
for (name in franchises$restaurant_chain)
{
    df_in_work$restaurant_chain[grepl(name,df_in_work$restaurant_chain)] <- name
    df_in_work$type[grepl(name,df_in_work$restaurant_chain)] <- "Franchises"
}

df_in_work$type[df_in_work$type==""] <- "SMBs"

##### 3.2 Extract location data from coordinates #####

- Using map.where() function to determine state and city information of the US based on coordinates
- Remove the ones without output as they are not locations in the United States
- the output of the function gives us state,city information, so we further split them into "state" and "city" columns

In [582]:
df_in_work$city <- map.where(database="county", df_in_work$longitude, df_in_work$latitude)
df_in_work <- subset(df_in_work, !is.na(city))
df_in_work <- separate(df_in_work,city,into = c("state","city"), sep=",")
head(df_in_work,20)

,restaurant_id,restaurant_chain,branch,platform,sub_platform,latitude,longitude,state,city,country,active,standardized_name,delivery_radius,geom,type
,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>
1,9744,loco coco,NA,caviar,,40.7650,-73.9660,new york,new york,US,TRUE,loco_coco,NA,NA,SMBs
2,4903,musubi,NA,caviar,,45.5046,-122.6438,oregon,multnomah,US,FALSE,musubi,NA,NA,SMBs
3,4723,the bakers' lounge,NA,caviar,,38.9246,-76.9901,district of columbia,washington,US,FALSE,the_bakers_lounge,NA,NA,SMBs
4,4128,parm,battery park city,caviar,,40.7142,-74.0156,new york,new york,US,TRUE,parm_-_battery_park_city,NA,NA,SMBs
5,2827,the plant cafe,california & pine - fidi,caviar,,37.7928,-122.3979,california,san francisco,US,FALSE,the_plant_cafe_-_california_pine_fidi,NA,NA,SMBs
6,4850,minton's,NA,caviar,,40.8047,-73.9523,new york,new york,US,FALSE,minton_s,NA,NA,SMBs
7,6301,brazilian bowl,albany park,caviar,,41.9686,-87.7087,illinois,cook,US,FALSE,brazilian_bowl_-_albany_park,NA,NA,SMBs
8,9006,gourmet palace china bistro,NA,caviar,,34.1645,-118.4143,california,los angeles,US,TRUE,gourmet_palace_china_bistro,NA,NA,SMBs
9,6208,liberty burger,keller,caviar,,32.9690,-96.8203,texas,dallas,US,TRUE,liberty_burger_-_keller,NA,NA,SMBs


### 4. Re-arrange Dataset & Perform Deduplication

In [595]:
df_clean <- df_in_work

In [596]:
filter(df_clean, (platform != sub_platform) & (sub_platform != "restaurant") & sub_platform != "")

restaurant_id,restaurant_chain,branch,platform,sub_platform,latitude,longitude,state,city,country,active,standardized_name,delivery_radius,geom,type
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>


In [621]:
subset(df_clean, is.na(platform) | platform == "")

restaurant_id,restaurant_chain,branch,platform,sub_platform,latitude,longitude,state,city,country,active,standardized_name,delivery_radius,geom,type
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>


##### 4.1 Remove unwanted columns and re-arrange column orders #####

In [597]:
df_final <- select(df_clean, -c("restaurant_id","geom","sub_platform", "standardized_name"))
col_order <- c('restaurant_chain','branch','type','active','platform','latitude','longitude','city','state','country','delivery_radius')
df_final <- df_final[,col_order]
head(df_final)

,restaurant_chain,branch,type,active,platform,latitude,longitude,city,state,country,delivery_radius
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,loco coco,NA,SMBs,TRUE,caviar,40.7650,-73.9660,new york,new york,US,NA
2,musubi,NA,SMBs,FALSE,caviar,45.5046,-122.6438,multnomah,oregon,US,NA
3,the bakers' lounge,NA,SMBs,FALSE,caviar,38.9246,-76.9901,washington,district of columbia,US,NA
4,parm,battery park city,SMBs,TRUE,caviar,40.7142,-74.0156,new york,new york,US,NA
5,the plant cafe,california & pine - fidi,SMBs,FALSE,caviar,37.7928,-122.3979,san francisco,california,US,NA
6,minton's,NA,SMBs,FALSE,caviar,40.8047,-73.9523,new york,new york,US,NA


##### 4.2 Perform deduplication #####

- To ensure each entry is unique, unique() function is required, and certain criteria need to be met:
    - A restaurant can have multiple delivery platform, despite them being active or not, so "platform" column is needed
    - A restaurant can have multiple locations, so coordinates variables are also needed
    - Same coordinates may have different restaurant names due to them being entered into the system during different time period, so "restaurant_chain" column is also needed
- After performing deduplication, about 85,031 entries were deleted as they are deemed repeated

In [604]:

dim(df)
dim(unique(df_final[c("restaurant_chain","platform","longitude","latitude")]))
dim(unique(df_final[c("restaurant_chain","longitude","latitude")]))

dim(df) - dim(unique(df_final[c("restaurant_chain","platform","longitude","latitude")]))


[1] 644089     13

[1] 559058      4

[1] 498516      3

[1] 85031     9

In [614]:
df_final_unique <- distinct(df_final, restaurant_chain,latitude,longitude,.keep_all = TRUE)
df_final <- distinct(df_final, restaurant_chain,platform, latitude,longitude,.keep_all = TRUE)

##### 4.3 Export the cleaned up dataset into a new csv file #####

In [615]:
write.csv(df_final_unique,"df_final_unique_V2.csv", row.names = TRUE)
write.csv(df_final,"df_final_V2.csv", row.names = TRUE)

In [617]:
dim(df_final_unique)
dim(df_final)

[1] 498516     11

[1] 559058     11

In [622]:
head(df_final_unique)

,restaurant_chain,branch,type,active,platform,latitude,longitude,city,state,country,delivery_radius
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,loco coco,NA,SMBs,TRUE,caviar,40.7650,-73.9660,new york,new york,US,NA
2,musubi,NA,SMBs,FALSE,caviar,45.5046,-122.6438,multnomah,oregon,US,NA
3,the bakers' lounge,NA,SMBs,FALSE,caviar,38.9246,-76.9901,washington,district of columbia,US,NA
4,parm,battery park city,SMBs,TRUE,caviar,40.7142,-74.0156,new york,new york,US,NA
5,the plant cafe,california & pine - fidi,SMBs,FALSE,caviar,37.7928,-122.3979,san francisco,california,US,NA
6,minton's,NA,SMBs,FALSE,caviar,40.8047,-73.9523,new york,new york,US,NA
